# Yelp API Calls
<p> In this project I am exploring the Yelp API to search my favorite city for cuisine types of my choice. At the end I will extract all results from my search and compile them into a dataframe. </p>

Import Modules

In [1]:
# Standard Imports
import pandas as pd
import os, json, math, os
from time import sleep
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

Load credentials

In [2]:
# Load API Creds
with open('creds.json') as file:
    login = json.load(file)
# Instantiate YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)

Environment Variables

In [3]:
LOCATION = 'Fresno, CA, 93720'
TERM = 'Pizza'
JSON_FILE = f"Data/results_in_progress_pizza.json"

Create progress file if it doesn't exist already and set n_results

In [4]:
# Set default value to 0 results and no previous results
n_results = 0
previous_results = []
if os.path.isfile(JSON_FILE) == False:
    # In progress file doesn't exist
    if len(os.path.dirname(JSON_FILE)) > 0:
        os.makedirs(os.path.dirname(JSON_FILE), exist_ok = True)
    # Tell user new file was made
    print(f"[!] {JSON_FILE} not found. Saving empty list to file.")
    with open(JSON_FILE, 'w') as file:
        json.dump([], file)
else:
    # In Progress does exist
    print(f"[!] {JSON_FILE} already exists.") 
    with open(JSON_FILE, 'r') as file:
        previous_results = json.load(file)
        n_results = len(previous_results)

print(f"[!] {n_results} previous results found.")

[!] Data/results_in_progress_pizza.json not found. Saving empty list to file.
[!] 0 previous results found.


Get total results of API Search

In [5]:
# add offset to our original api call
results = yelp_api.search_query(location=LOCATION,
                                      term=TERM,
                                      offset = n_results)
# Get data on the query size
total_results = results['total']
results_per_page = len(results['businesses'])

In [6]:
# Find out total pages
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
print(f"{int(n_pages)} pages with a total of {total_results} results")

14 pages with a total of 266 results


Add the current results to the JSON, then loop through the rest.

In [7]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as file:
     json.dump(previous_results, file)

In [8]:
# Not using TQDM - issues with mac m1
for i in range(1, n_pages + 1):
    sleep(.2)
    # check results length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    # save number of results for to use as offset
    n_results = len(previous_results)
    # use n_results as the OFFSET 
    results = yelp_api.search_query(location = LOCATION,
                                    term = TERM, 
                                    offset = n_results + 1)    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    with open(JSON_FILE,'w') as file:
        json.dump(previous_results, file)

Turn json into DataFrame

In [9]:
df = pd.read_json(JSON_FILE)
display(df.head(), df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,UVW_GF1zGsKN8OBOTGQUGA,annesso-pizzeria-fresno-2,Annesso Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/gT-mop...,False,https://www.yelp.com/biz/annesso-pizzeria-fres...,197,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 36.85922376493786, 'longitude': -...",[],$$,"{'address1': '8484 N Friant Rd', 'address2': '...",+15599818995,(559) 981-8995,2036.953801
1,8OlqbBzTXpySzmjnQttF2w,premo-pizza-fresno,Premo Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/Je-uNp...,False,https://www.yelp.com/biz/premo-pizza-fresno?ad...,231,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 36.8650556, 'longitude': -119.738...","[pickup, delivery]",$$,"{'address1': '8887 N Chestnut Ave', 'address2'...",+15592981122,(559) 298-1122,1877.710131
2,nqBZ24HuLGIlFOy972UqCg,mountain-mikes-pizza-fresno-7,Mountain Mike's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/TAcGpe...,False,https://www.yelp.com/biz/mountain-mikes-pizza-...,52,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 36.8659608, 'longitude': -119.758...","[pickup, delivery]",$$,"{'address1': '8965 N Cedar Ave', 'address2': '...",+15592614111,(559) 261-4111,286.458294
3,R9NkF0TszWbalQ_OU7d6wA,bombay-pizza-fresno-3,Bombay Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/UeUsl7...,False,https://www.yelp.com/biz/bombay-pizza-fresno-3...,99,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 36.85155, 'longitude': -119.77542}","[pickup, delivery]",NaN,"{'address1': '7797 N First St', 'address2': No...",+15594493281,(559) 449-3281,1934.213621
4,minzlReeKFG7xaPABJJrxw,marco-s-pizza-opening-soon-clovis-3,Marco's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/-4a41r...,False,https://www.yelp.com/biz/marco-s-pizza-opening...,3,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 36.84628, 'longitude': -119.72914}","[pickup, delivery]",NaN,"{'address1': '865 N Willow Ave', 'address2': '...",+15597538700,(559) 753-8700,3522.246882


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
260,c8x_8uc0nIrwGUObuvouvg,casa-corona-fresno-2,Casa Corona,https://s3-media3.fl.yelpcdn.com/bphoto/Q3ZSUR...,False,https://www.yelp.com/biz/casa-corona-fresno-2?...,232,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.0,"{'latitude': 36.823191, 'longitude': -119.824777}",[delivery],$$,"{'address1': '1724 W Bullard Ave', 'address2':...",+15592838605,(559) 283-8605,7301.667065
261,ECiZyn62_HdjkycEIYy61w,7-eleven-clovis,7-Eleven,https://s3-media4.fl.yelpcdn.com/bphoto/Y_KTHZ...,False,https://www.yelp.com/biz/7-eleven-clovis?adjus...,9,"[{'alias': 'convenience', 'title': 'Convenienc...",3.5,"{'latitude': 36.8161107, 'longitude': -119.700...","[pickup, delivery]",$,"{'address1': '1170 Clovis Ave', 'address2': No...",+15592981277,(559) 298-1277,7621.124146
262,Mgo2YhEDP7h7Wdhes2YZYw,subway-clovis-29,Subway,https://s3-media1.fl.yelpcdn.com/bphoto/4dNvyK...,False,https://www.yelp.com/biz/subway-clovis-29?adju...,15,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",2.0,"{'latitude': 36.8370793783544, 'longitude': -1...","[pickup, delivery]",$,{'address1': '108 Herndon Ave Toll House Cross...,+15593250504,(559) 325-0504,8326.420136
263,UZRqgFkifEtZjHrJ_0ogBQ,subway-clovis-32,Subway,https://s3-media3.fl.yelpcdn.com/bphoto/vhWsc0...,False,https://www.yelp.com/biz/subway-clovis-32?adju...,16,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",3.0,"{'latitude': 36.8090540427663, 'longitude': -1...","[pickup, delivery]",$,"{'address1': '1195 Shaw Ave', 'address2': 'Ste...",+15592991892,(559) 299-1892,8621.612938
264,HkN6cx0EWOrnLIa4YCrnmg,subway-clovis-23,Subway,https://s3-media2.fl.yelpcdn.com/bphoto/cMl2FJ...,False,https://www.yelp.com/biz/subway-clovis-23?adju...,22,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",1.5,"{'latitude': 36.7932950298325, 'longitude': -1...","[pickup, delivery]",$,"{'address1': '1830 Ashlan Ave', 'address2': ''...",+15592924833,(559) 292-4833,10673.979239


Save to CSV

In [10]:
df.to_csv('Data/final_results.csv.gz', compression = 'gzip', index = False)